In [8]:
import json
import pandas as pd
import numpy as np

import requests
from flask_restx import Model
from pydantic import BaseModel
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report

Доступные ручки:

In [9]:
info_endpoint = 'http://127.0.0.1:5000/test_api/ml_models'
train_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/train'
delete_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/delete'
predict_endpoint = 'http://127.0.0.1:5000/test_api/ml_models/predict'

Посмотрим, как работает созданное API. Для этого запустим файл `my_api.py` и покидаем в ручки запросы. Первая ручка `info_endpoint` - с информацией о доступных моделях и гиперпараметрах

In [10]:
print(requests.get('http://127.0.0.1:5000/test_api/ml_models').json())

here is a list of available models:
 - Logistic Regression, allowed hyperparameters: {'C': 'float (>0)'};
 - Random Forest, allowed hyperparameters: {'n_estimators': 'int (>0)', 'max_depth': 'int (>0)'};


Создадим пару тестовых датасетов

In [11]:
x_1, y_1 = make_classification(n_samples=400, n_features=5, n_classes=2)

dataset_train_1 = pd.DataFrame(x_1[:100, :])
dataset_train_1['target'] = y_1[:100]
dataset_train_1 = dataset_train_1.to_json()

dataset_train_2 = pd.DataFrame(x_1[100:300, :])
dataset_train_2['target'] = y_1[100:300]
dataset_train_2 = dataset_train_2.to_json()

dataset_test = pd.DataFrame(x_1[300:, :])
dataset_test = dataset_test.to_json()

Получаем из ручки `train_endpoint` информацию об обученных моделях - их нет

In [12]:
requests.get(train_endpoint).json()

{}

Отправим запрос с обучением случайного леса с заданными гиперпараметрами; в ответ получаем небольшой отчет об основных метриках обучения на 5 фолдах 

In [21]:
json_train_data = {
    'model_name': 'rfc',
    'model_type': 'Random Forest',
    'hyperparams': {'n_estimators': 200, 'max_depth': 10},
    'train_data': dataset_train_1,
}

r = requests.post(train_endpoint, json=json_train_data)
print(r.json())

{"score":{"precision":0.936,"recall":0.933,"roc_auc":0.983,"f1_score":0.93}}


Проверим, что в списке появилась обученная модель

In [22]:
requests.get(train_endpoint).json()

{'rfc': 'RandomForestClassifier(max_depth=10, n_estimators=200)'}

Отправим теперь `dataset_train_2`, на котором переобучим созданную `rfc` модель

In [24]:
json_train_data = {
    'model_name': 'rfc',
    'new_data': dataset_train_2,
}

r = requests.put(train_endpoint, data=json_train_data)
print(r.json())

{"score":{"precision":0.964,"recall":0.94,"roc_auc":0.997,"f1_score":0.95}}


Проверим, что обученная модель работает - дадим ей на вход оставшуюся часть датасета и посмотрим на `classification_report`

In [30]:
json_pred_data = {
    'model_name': 'rfc',
    'predict_data': dataset_test,
}

r = requests.post(predict_endpoint, json=json_pred_data)
y_pred = r.json()
print(classification_report(y_1[300:], y_pred))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96        46
           1       0.96      0.96      0.96        54

    accuracy                           0.96       100
   macro avg       0.96      0.96      0.96       100
weighted avg       0.96      0.96      0.96       100



Напоследок удалим созданную модель с помощью ручки `delete_endpoint` и выведем список обученных моделей

In [32]:
r = requests.delete(delete_endpoint + '/' + 'rfc')
requests.get(train_endpoint).json()

{}